# BLAST



Dealing with BLAST can be split up into two steps, both of which can be done from within Biopython.
Firstly, running BLAST for your query sequence(s), and getting some output. Secondly, parsing the BLAST
output in Python for further analysis.

Your first introduction to running BLAST was probably via the NCBI web-service. In fact, there are
lots of ways you can run BLAST, which can be categorised in several ways. The most important distinction
is running BLAST locally (on your own machine), and running BLAST remotely (on another machine,
typically the NCBI servers). The former will not be part of this Python course, however we will run BLAST from within a Python script in this chapter. 

## 7.1 Running BLAST over the internet

We use the function `qblast()` in the `Bio.Blast.NCBIWWW` module to call the online version of BLAST. This
has three non-optional arguments:
- The first argument is the blast program to use for the search, as a lower case string. The options and descriptions of the programs are available at https://blast.ncbi.nlm.nih.gov/Blast.cgi. Currently qblast only works with blastn, blastp, blastx, tblast and tblastx.
- The second argument specifies the databases to search against. Again, the options for this are available on the NCBI Guide to BLAST ftp://ftp.ncbi.nlm.nih.gov/pub/factsheets/HowTo_BLASTGuide.pdf.
- The third argument is a string containing your query sequence. This can either be the **sequence** itself, the **sequence in fasta** format, or an **identifier like a GI number**.

The `qblast` function also takes a number of other option arguments which are basically analogous to the
different parameters you can set on the BLAST web page. We'll just highlight a few of them here:

- The argument `url_base` sets the base URL for running BLAST over the internet. By default it connects to the NCBI, but one can use this to connect to an instance of NCBI BLAST running in the cloud. 
- The `qblast` function can return the BLAST results in various formats, which you can choose with the optional format_type keyword: "HTML", "Text", "ASN.1", or "XML". The default is "XML", as that is the format expected by the parser, described in section 7.3 below.
- The argument `expect` sets the expectation or *e-value* threshold.

In [1]:
from Bio.Blast import NCBIWWW
help(NCBIWWW.qblast)

Help on function qblast in module Bio.Blast.NCBIWWW:

qblast(program, database, sequence, url_base='https://blast.ncbi.nlm.nih.gov/Blast.cgi', auto_format=None, composition_based_statistics=None, db_genetic_code=None, endpoints=None, entrez_query='(none)', expect=10.0, filter=None, gapcosts=None, genetic_code=None, hitlist_size=50, i_thresh=None, layout=None, lcase_mask=None, matrix_name=None, nucl_penalty=None, nucl_reward=None, other_advanced=None, perc_ident=None, phi_pattern=None, query_file=None, query_believe_defline=None, query_from=None, query_to=None, searchsp_eff=None, service=None, threshold=None, ungapped_alignment=None, word_size=None, short_query=None, alignments=500, alignment_view=None, descriptions=500, entrez_links_new_window=None, expect_low=None, expect_high=None, format_entrez_query=None, format_object=None, format_type='XML', ncbi_gi=None, results_file=None, show_overview=None, megablast=None, template_type=None, template_length=None)
    BLAST search using NCBI's

Note that the default settings on the NCBI BLAST website are not quite the same as the defaults
on QBLAST. If you get different results, you'll need to check the parameters (e.g., the expectation value
threshold and the gap values).

For example, if you have a nucleotide sequence you want to search against the nucleotide database (nt)
using BLASTN, and you know the GI number of your query sequence, you can use:

In [2]:
# Takes a long time to run
result_handle = NCBIWWW.qblast("blastn", "nt", "8332116")
result_handle

Alternatively, if we have our query sequence already in a FASTA formatted file, we just need to open the
file and read in this record as a string, and use that as the query argument:

In [4]:
with open("data/NC_005816.fna") as fh:
    fasta_string = fh.read()
    result_handle = NCBIWWW.qblast("blastn", "nt", fasta_string)
result_handle

We could also have read in the FASTA file as a SeqRecord and then supplied just the sequence itself:

In [5]:
from Bio import SeqIO
record = SeqIO.read("data/NC_005816.fna", format="fasta")
result_handle = NCBIWWW.qblast("blastn", "nt", record.seq)
result_handle

Supplying just the sequence means that BLAST will assign an identifier for your sequence automatically.
You might prefer to use the `SeqRecord object`'s *format method* to make a FASTA string (which will include
the existing identifier):

In [ ]:
record = SeqIO.read("data/NC_005816.fna", format="fasta")
result_handle = NCBIWWW.qblast("blastn", "nt", record.format("fasta"))
result_handle

This approach makes more sense if you have your sequence(s) in a non-FASTA file format which you can
extract using `Bio.SeqIO`.
Whatever arguments you give the `qblast()` function, you should get back your results in a handle
object (by default in XML format). The next step would be to parse the XML output into Python objects
representing the search results (Section 7.3), but you might want to save a local copy of the output file first.

This is especially useful when debugging code that extracts info from the BLAST results (because
re-running the online search is slow and wastes the NCBI computer time).

**We need to be a bit careful since we can use `result_handle.read()` to read the BLAST output only
once - calling `result_handle.read()` again returns an empty string.**

In [6]:
with open("data/my_blast.xml", "w") as out_handle:
    out_handle.write(result_handle.read())

After doing this, the results are in the file `my_blast.xml` and the original handle has had all its data
extracted (so we closed it). However, the parse function of the BLAST parser (described in 7.3) takes a
file-handle-like object, so we can just open the saved file for input:

In [ ]:
with open("data/my_blast.xml", 'r') as fh:
    my_blast = fh.read()
    
#print(my_blast)

Now that we've got the BLAST results back into a handle again, we are ready to do something with
them, so this leads us right into the parsing section.

## 7.3 Parsing BLAST output
BLAST can generate output in various formats, such as XML, HTML and plain text. It is however highly recommended to work with the XML format. Not only is the XML output more stable than the plain text and HTML output, it is also much easier to parse automatically, making Biopython a whole lot more stable.

You can get BLAST output in XML format in various ways. For the parser, it doesn't matter how the output was generated, as long as it is in the XML format. You then need to get a handle to the results. 

Let's fetch some data

In [15]:
# Uncomment following lines if you haven't run them above
#from Bio.Blast import NCBIWWW
result_handle = NCBIWWW.qblast("blastn", "nt", "8332116")

In [7]:
result_handle

Once we have a handle to the BLAST output (in XML format), we can either parse a single result or multiple results. 

Just like `Bio.SeqIO` and `Bio.AlignIO`, we have a pair of input functions, read
and parse, where read is for when you have exactly one object, and parse is an iterator for when you can
have lots of objects. The result of this reading/parsing process is/are a BLAST
record objects.

In [8]:
from Bio.Blast import NCBIXML

In [ ]:
# For a single BLAST result
#blast_record = NCBIXML.read(result_handle)

In [16]:
# For multiple BLAST results in one XML file
blast_records = NCBIXML.parse(result_handle)
blast_records

<generator object parse at 0x0000021049DE6750>

In order to iterate over the objects that are part of the `blast_records`, we can use the following:

# This i did myself

In [23]:
with open("data/my_blast.xml") as result_handle:
    blast_records = NCBIXML.parse(result_handle)
    for record in blast_records:
        # Each record has 
        for descr in record.descriptions:
            print(descr.title)
            print(descr.e)
            print(descr.score)

gi|45357364|gb|AE017046.1| Yersinia pestis biovar Microtus str. 91001 plasmid pPCP1, complete sequence
gi|311902116|gb|HM807366.1| Yersinia pestis strain C790 plasmid pPCP1, complete sequence
gi|262363963|gb|CP001588.1| Yersinia pestis D106004 plasmid pPCY1, complete sequence
gi|1518099693|gb|CP033698.1| Yersinia pestis strain FDAARGOS_601 plasmid unnamed2, complete sequence
gi|1046916732|gb|CP016276.1| Yersinia pestis strain Cadman plasmid pPCP1, complete sequence
gi|294352540|gb|CP001596.1| Yersinia pestis Z176003 plasmid pPCP1, complete sequence
gi|5763810|emb|AL109969.1| Yersinia pestis CO92 plasmid pPCP1
gi|2996216|gb|AF053945.1| Yersinia pestis strain KIM5 plasmid pPCP1, complete sequence >gi|311902126|gb|HM807367.1| Yersinia pestis strain C2614 plasmid pPCP1, complete sequence >gi|311902136|gb|HM807368.1| Yersinia pestis strain C2944 plasmid pPCP1, complete sequence
gi|162350793|gb|CP000900.1| Yersinia pestis Angola plasmid pMT-pPCP, complete sequence
gi|262364123|gb|CP001592.1|